# Image classification for Medical Image

以下將演示如何使用 huggingface 框架並使用其中提供的函數載入自定義的資料集，以達到影像分類的結果。

huggingface 的工作流程：
![](https://hackmd.io/_uploads/HkEzgZdwh.png)

In [ ]:
# 安裝所需套件
!pip -q install transformers==4.30.0 datasets evaluate accelerate

In [ ]:
# 匯入相關套件
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

## Load dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("/home/jovyan/ta-shared-ii/datas/new_NIH_data_1800/")

In [ ]:
dataset

In [ ]:
train_dataset = dataset['train']
valid_dataset = dataset['validation']

In [ ]:
item = next(iter(train_dataset))

In [ ]:
print(item['label'])
item['image']

In [ ]:
item['image'].size

In [ ]:
train_dataset.features["label"].names

In [ ]:
labels = train_dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [ ]:
labels

現在，按照標籤 id 轉換成名稱：

In [ ]:
id2label[1]

## Preprocess


接下來的步驟是載入指定模型使用的影像處理器，將影像處理成張量：

In [ ]:
from transformers import AutoImageProcessor

# checkpoint = "google/vit-base-patch16-224-in21k"  # model name
# checkpoint = "google/efficientnet-b6"
checkpoint = "microsoft/cvt-13"
# checkpoint = "microsoft/resnet-50"
# checkpoint = "facebook/convnext-tiny-224"
# checkpoint = "facebook/convnext-base-224"
# # checkpoint = "facebook/convnext-large-224"

image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
image_processor

---
將影像進行轉換，使模型更具一般性以應付過擬合的情況。這裡會使用的 torchvision 中 transforms 的模組，但也能替換成其他適用的影像處理套件。

將其調整影像大小以及隨機仿射處理，並使用影像的平均值和標準差進行標準化：

In [ ]:
from torchvision.transforms import Resize, RandomAffine, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([Resize(size),
#                        RandomHorizontalFlip(),
                       RandomAffine(degrees=10, scale=(0.9, 1.1)),
                       ToTensor(),
                       normalize])

接下來創建一個預處理函數，轉換並回傳影像的像素值作為模型的輸入：

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

要在整個資料集上應用預處理函數，可以使用 Hugging Face 資料集的 [with_transform](https://huggingface.co/docs/datasets/v2.11.0/en/package_reference/main_classes#datasets.Dataset.with_transform) 方法。當載入資料集的一個元素時，轉換會即時套用：

In [ ]:
train_ds = train_dataset.with_transform(transforms)
valid_ds = valid_dataset.with_transform(transforms)

In [ ]:
import matplotlib.pyplot as plt
import torch

In [ ]:
item = next(iter(train_ds))

In [ ]:
item

In [ ]:
print(item['label'])
print(item['pixel_values'].size())
plt.imshow(torch.permute(item['pixel_values'], (1, 2, 0)))

現在使用 [DefaultDataCollator](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/data_collator#transformers.DefaultDataCollator) 創建一個批次樣本。與 Hugging face 裡 Transformers 的其他資料收集器不同，DefaultDataCollator 不會套用額外的預處理，例如填充（padding）。

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

## Evaluate

在訓練過程中加入評估指標通常有助於評估模型的表現。可以使用 Hugging Face 的 [Evaluate](https://huggingface.co/docs/evaluate/index) 函式庫快速載入評估方法。在此任務上載入 [accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy) 指標（請參閱 Hugging Face 的 Evaluate [快速導覽](https://huggingface.co/docs/evaluate/a_quick_tour)，以了解如何載入和計算指標的詳細資訊）：

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

然後創建一個函數，將預測及標籤使用 [compute](https://huggingface.co/docs/evaluate/v0.4.0/en/package_reference/main_classes#evaluate.EvaluationModule.compute) 以計算準確度：

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

現在已準備好開始訓練模型了！使用 [AutoModelForImageClassification](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/auto#transformers.AutoModelForImageClassification) 載入模型。指定標籤的數量以及標籤的對應方式：

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True  # 預訓練模型的分類數量與自定義的資料集不同時使用
)

接著的階段，只剩以下三個步驟：

1. 在 [TrainingArguments](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.TrainingArguments) 中定義訓練的超參數。請務必留意資料集中未使用的資訊，設定 remove_unused_columns=False 可以防止被刪除未使用到的資訊！例如 image，這會導致無法獲得 pixel_values。另一個必需設定的參數是 output_dir，指定模型儲存的位置。通過設定 push_to_hub=True 將模型上傳至 Hub（需要登入 Hugging Face 才能上傳模型）。在每個 epoch 結束時，[Trainer](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer) 將評估準確性並儲存訓練模型。
2. 將訓練參數、模型、資料集、預處理器、資料收集器以及計算評估指標函數傳遞給 [Trainer](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer)。
3. 呼叫 [train](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer.train) 來微調模型。

In [ ]:
# freeze layers without training
for param in model.cvt.encoder.stages[:1].parameters():
    param.requires_grad = False

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="my_cvt_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy")

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

In [ ]:
from transformers.integrations import MLflowCallback
trainer.remove_callback(MLflowCallback)
trainer.train()

## Visualization 

In [ ]:
train_loss = []
valid_loss = []
valid_acc = []
for i in range(0, len(trainer.state.log_history)-1, 2):
    train_loss.append(trainer.state.log_history[i]['loss'])
    valid_loss.append(trainer.state.log_history[i+1]['eval_loss'])
    valid_acc.append(trainer.state.log_history[i+1]['eval_accuracy'])

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
plt.plot(range(len(train_loss)), train_loss, label='train')
plt.plot(range(len(valid_loss)), valid_loss, label='val')
plt.legend()
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Cross entropy')
plt.subplot(1, 2, 2)
plt.plot(range(len(valid_acc)), valid_acc)
plt.title('Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
cfm_metric = evaluate.load("BucketHeadP65/confusion_matrix")

In [ ]:
trainer.evaluate(eval_dataset=valid_ds)

In [ ]:
outputs = trainer.predict(test_dataset=valid_ds)
cm = cfm_metric.compute(predictions=np.argmax(outputs.predictions, axis=1),
                        references=outputs.label_ids)['confusion_matrix']

In [ ]:
import seaborn as sns
plt.figure(figsize=(6, 4))
ax = sns.heatmap(cm, annot=True)
ax.set(xlabel="prediction", ylabel="label")
ax.set_xticklabels(labels)
ax.set_yticklabels(labels, rotation=0)
plt.show()

## Inference

現在，微調後的模型以存放在指定路徑，並可使用它來進行推論！

載入想要進行推論的影像：

In [ ]:
test_dataset = dataset['test']
image = test_dataset['image'][0]
label = test_dataset['label'][0]

In [ ]:
label

In [ ]:
id2label[label]

In [ ]:
image

使用微調後的模型進行推論最簡單的方法是在 pipline() 中設定。藉由指定的模型建構一個影像分類的 pipeline，然後將影像傳遞給它：

In [ ]:
from transformers import pipeline

classifier = pipeline("image-classification", model="my_cvt_model/checkpoint-380/")
classifier(image)

載入影像處理器對影像進行預處理，並以 PyTorch 的張量型態回傳作為輸入：

In [ ]:
from transformers import AutoImageProcessor
import torch

image_processor = AutoImageProcessor.from_pretrained("my_cvt_model/checkpoint-380/")
inputs = image_processor(image.convert('RGB'), return_tensors="pt")

In [ ]:
inputs

將輸入傳遞給模型，並回傳 logits（尚未經過 softmax）：

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    "my_cvt_model/checkpoint-380/")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]

* ### Evaluate

In [ ]:
test_ds = test_dataset.with_transform(transforms)

In [ ]:
trainer.evaluate(eval_dataset=test_ds)

In [ ]:
outputs = trainer.predict(test_dataset=test_ds)
cm = cfm_metric.compute(predictions=np.argmax(outputs.predictions, axis=1),
                        references=outputs.label_ids)['confusion_matrix']

In [ ]:
import seaborn as sns
plt.figure(figsize=(6, 4))
ax = sns.heatmap(cm, annot=True)
ax.set(xlabel="prediction", ylabel="label")
ax.set_xticklabels(labels)
ax.set_yticklabels(labels, rotation=0)
plt.show()

---

## Explainable AI

可解釋性人工智慧（Explainable AI）是指能夠解釋和解釋其結論、推論和決策過程的人工智慧系統。傳統上，許多機器學習和深度學習模型往往被視為黑盒，它們可以對數據進行複雜的計算，但很難理解為什麼做出特定的預測或決策。

其中 GradCAM（Gradient-weighted Class Activation Mapping）用於視覺任務中對深度學習模型的解釋。它可以幫助我們理解模型在圖像分類或物體檢測等任務中的決策過程，並提供對模型預測的視覺化解釋。

![](https://hackmd.io/_uploads/Syf544uP3.png)

以下是使用 GradCAM 進行可解釋性 AI 的基本步驟：

1. 準備模型：首先，需要準備一個已經訓練好的深度學習模型，例如卷積神經網絡（CNN）用於圖像分類。
2. 特徵提取：通過將圖像輸入模型，獲得中間層的特徵圖。這些特徵圖是在網絡中不同層次上提取的高級視覺特徵。
3. 梯度計算：對於目標類別，計算反向傳播的梯度。這些梯度表示目標類別相對於特徵圖的重要性。
4. 特徵加權：將梯度與特徵圖進行加權相乘，以強調對目標類別貢獻最大的區域。
5. 生成顯著性圖：將加權特徵圖進行空間平均或加總，得到一個顯著性圖，表示模型對於不同區域的關注程度。
6. 視覺化解釋：將顯著性圖與原始圖像進行融合或覆蓋，生成一張視覺化的解釋圖。這個解釋圖可以顯示出模型在預測中關注的重要區域。

In [ ]:
# 安裝支援 PyTorch 的 GradCAM 套件
!pip -q install grad-cam

In [ ]:
# 匯入相關套件
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from typing import List, Callable, Optional
from PIL import Image
import cv2

from torchvision.transforms import ToPILImage
from functools import partial

In [ ]:
image = dataset['test']["image"][0]
label = dataset['test']['label'][0]

img_tensor = Compose([Resize(size), ToTensor(), normalize])(image.convert("RGB"))

In [ ]:
label

In [ ]:
img_tensor.shape

In [ ]:
image

In [ ]:
# 按照不同的模型選定 reshape 的方式
def reshape_transform_cvt_huggingface(tensor, model, width, height):
    tensor = tensor[:, 1 :, :]  # remove 1 cls token
    tensor = tensor.reshape(tensor.size(0),
                            height,
                            width,
                            tensor.size(-1))
    
    # https://github.com/huggingface/transformers/blob/a2c90a7f7b1f8a2a8217c962a04a1a65638121d5/src/transformers/models/cvt/modeling_cvt.py#L699
    norm = model.layernorm(tensor)
    return norm.transpose(2, 3).transpose(1, 2)  # (Batch, features, height, width)

In [ ]:
def reshape_gradcam_transform_cvt_huggingface(tensor, model, width, height):
    tensor = tensor[:, 1 :, :]  # remove 1 cls token
    tensor = tensor.reshape(tensor.size(0),
                            height,
                            width,
                            tensor.size(-1))
    return tensor.transpose(2, 3).transpose(1, 2)  # (Batch, features, height, width)

In [ ]:
# 拿取模型輸出的 logits
class HuggingfaceToTensorModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(HuggingfaceToTensorModelWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model(x).logits

In [ ]:
def run_grad_cam_on_image(model, #: torch.nn.Module,
                          target_layer, #: torch.nn.Module,
                          targets_for_gradcam, #: List[Callable],
                          reshape_transform, #: Optional[Callable],
                          input_tensor=img_tensor, #: torch.nn.Module=img_tensor,
                          input_image=image, #: Image=image,
                          method=GradCAM): #: Callable=GradCAM):
    with method(model=HuggingfaceToTensorModelWrapper(model),
                 target_layers=[target_layer],
                 reshape_transform=reshape_transform) as cam:

        # Replicate the tensor for each of the categories we want to create Grad-CAM for:
        repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

        batch_results = cam(input_tensor=repeated_tensor,
                            targets=targets_for_gradcam)
        results = []
        for grayscale_cam in batch_results:
            # convert graycam size to input image size
            grayscale_cam = cv2.resize(grayscale_cam, input_image.size) 
            
            visualization = show_cam_on_image(np.float32(input_image)/255,
                                              grayscale_cam,
                                              use_rgb=True)
            # Make it weight less in the notebook:
            visualization = cv2.resize(visualization,
                                       (visualization.shape[1]//2, visualization.shape[0]//2))
            results.append(visualization)
        return np.hstack(results)

---

選定一張影像作為演示

In [ ]:
targets_for_gradcam = [ClassifierOutputTarget(label2id['Atelectasis']),
                       ClassifierOutputTarget(label2id['Effusion']),
                       ClassifierOutputTarget(label2id['NoFinding'])]

In [ ]:
reshape_transform_gradcam = partial(reshape_gradcam_transform_cvt_huggingface,
                            model=model,
                            width=img_tensor.shape[2]//16,
                            height=img_tensor.shape[1]//16)

* ### 如何知道應該選擇哪個目標層？**選擇在分類器或池化層之前的最後一層**

In [ ]:
model

In [ ]:
target_layer_gradcam = model.cvt.encoder.stages[-1].layers[-2]

In [ ]:
display(Image.fromarray(run_grad_cam_on_image(model=model,
                                              target_layer=target_layer_gradcam,
                                              targets_for_gradcam=targets_for_gradcam,
                                              reshape_transform=reshape_transform_gradcam,
                                              input_image=image.convert('RGB'),
                                              input_tensor=img_tensor)))

---
ScoreCAM（Score-based Class Activation Mapping）和 GradCAM（Gradient-weighted Class Activation Mapping）都是解釋卷積神經網絡（CNN）模型的可視化方法，用於可視化模型對圖像中不同區域的關注程度。它們的主要區別在於**權重的計算方式**和**解釋的焦點**。

In [ ]:
from pytorch_grad_cam import ScoreCAM

In [ ]:
display(Image.fromarray(run_grad_cam_on_image(model=model,
                                              target_layer=target_layer_gradcam,
                                              targets_for_gradcam=targets_for_gradcam,
                                              reshape_transform=reshape_transform_gradcam,
                                              input_image=image.convert('RGB'),
                                              input_tensor=img_tensor,
                                              method=ScoreCAM)))

* Reference tutorial: https://jacobgil.github.io/pytorch-gradcam-book/HuggingFace.html